In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix, mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor,GradientBoostingRegressor
from sklearn.svm import LinearSVC

In [4]:
from scipy.sparse import hstack
import joblib

In [5]:
df = pd.read_json("data/problems_data.jsonl",lines=True)
df.head()

,title,description,input_description,output_description,sample_io,problem_class,problem_score,url
0,Uuu,Unununium (Uuu) was the name of the chemical\n...,The input consists of one line with two intege...,The output consists of $M$ lines where the $i$...,"[{'input': '7 10', 'output': '1 2 2 3 1 3 3 4 ...",hard,9.7,https://open.kattis.com/problems/uuu
1,House Building,A number of eccentrics from central New York h...,"The input consists of $10$ test cases, which a...",Print $K$ lines with\n the positions of the...,"[{'input': '0 2 3 2 50 60 50 30 50 40', 'outpu...",hard,9.7,https://open.kattis.com/problems/husbygge
2,Mario or Luigi,Mario and Luigi are playing a game where they ...,,,"[{'input': '', 'output': ''}]",hard,9.6,https://open.kattis.com/problems/marioorluigi
3,The Wire Ghost,Žofka is bending a copper wire. She starts wit...,The first line contains two integers $L$ and $...,The output consists of a single line consistin...,"[{'input': '4 3 3 C 2 C 1 C', 'output': 'GHOST...",hard,9.6,https://open.kattis.com/problems/thewireghost
4,Barking Up The Wrong Tree,"Your dog Spot is let loose in the park. Well, ...",The first line of input consists of two intege...,Write a single line containing the length need...,"[{'input': '2 0 10 0 10 10', 'output': '14.14'...",hard,9.6,https://open.kattis.com/problems/barktree


In [6]:
df.shape
df.columns
df.head(2)

,title,description,input_description,output_description,sample_io,problem_class,problem_score,url
0,Uuu,Unununium (Uuu) was the name of the chemical\n...,The input consists of one line with two intege...,The output consists of $M$ lines where the $i$...,"[{'input': '7 10', 'output': '1 2 2 3 1 3 3 4 ...",hard,9.7,https://open.kattis.com/problems/uuu
1,House Building,A number of eccentrics from central New York h...,"The input consists of $10$ test cases, which a...",Print $K$ lines with\n the positions of the...,"[{'input': '0 2 3 2 50 60 50 30 50 40', 'outpu...",hard,9.7,https://open.kattis.com/problems/husbygge


In [7]:
df["full_text"] = ( df["title"].fillna("") + " " +  df["description"].fillna("") + " " + 
                    df["input_description"].fillna("") + " " + df["output_description"].fillna("") + " " + df["sample_io"].astype(str).fillna(""))

In [8]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9+\-*/=<> ]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

df["clean_text"]=df["full_text"].apply(clean_text)

In [9]:
KEYWORDS = [ "dp","dynamic programming", "graph", "tree", "bfs", "dfs", "recursion", "backtracking", "greedy", "binary search", "segment tree", 
             "bitmask", "math", "probability" ]

def keyword_features(text):
    return [text.count(k) for k in KEYWORDS]

def numeric_features(text):
    return [len(text), sum(c.isdigit() for c in text), sum(c in "+-*/=<>" for c in text)]

In [10]:
keyword_matrix = np.array(df["clean_text"].apply(keyword_features).tolist())

numeric_matrix = np.array(df["clean_text"].apply(numeric_features).tolist())                           

In [11]:
vectorizer = TfidfVectorizer( analyzer = "char", max_features=3000, ngram_range=(1,2), min_df=3 )
X_tfidf = vectorizer.fit_transform(df["clean_text"])

In [12]:
X_final = hstack([X_tfidf, keyword_matrix, numeric_matrix])

In [13]:
y_class = df["problem_class"]

X_train, X_test, y_train, y_test =train_test_split(X_final, y_class, test_size=0.2, random_state=42, stratify=y_class)

In [14]:
classification_models = {"Logistic Regression": LogisticRegression(max_iter=2000, class_weight ="balanced"),
                         "Linear SVM": LinearSVC(max_iter = 5000, class_weight = "balanced"),
                         "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42) }

In [15]:
from sklearn.metrics import balanced_accuracy_score
clf_results = []

for name, model in classification_models.items():
    model.fit(X_train,y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test,preds)
    bal_acc = balanced_accuracy_score(y_test, preds)
    clf_results.append((name,acc,bal_acc))

C:\Users\User\AppData\Roaming\Python\Python313\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 2000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=2000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [16]:
clf_results_df = pd.DataFrame(clf_results, columns=["Model","Accuracy","Balanced Accuracy"])
clf_results_df

,Model,Accuracy,Balanced Accuracy
0,Logistic Regression,0.419198,0.454170
1,Linear SVM,0.479951,0.459057
2,Random Forest,0.507898,0.403733


In [18]:
df["problem_class"].value_counts(normalize=True)


problem_class
hard      0.472033
medium    0.341683
easy      0.186284
Name: proportion, dtype: float64

#### Among the classifiers,Logistic Regression model the best balanced accuracy indicating uniform performace across classes.
#### Random Forest model shows the best accuracy (approx. 51%) and it outperforms the baseline accuracy(approx. 47%). Hence, it is chosen as the final classifier.

In [18]:
y_score = df["problem_score"]

X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_final, y_score, test_size= 0.2, random_state=42)

In [19]:
cm = confusion_matrix(
    y_test,
    preds,
    labels=["easy", "medium", "hard"]
)

cm_df = pd.DataFrame(
    cm,
    index=["Actual Easy", "Actual Medium", "Actual Hard"],
    columns=["Pred Easy", "Pred Medium", "Pred Hard"]
)

cm_df

,Pred Easy,Pred Medium,Pred Hard
Actual Easy,23,28,102
Actual Medium,8,46,227
Actual Hard,9,31,349


In [20]:
from sklearn.linear_model import Ridge
regression_models = { "Linear Regression": LinearRegression(),  "Ridge Regression": Ridge(alpha=1.0)
                      #,"Random Forest Regressor": RandomForestRegressor(n_estimators=200,random_state=42),
                      #"Gradient Boosting Regressor": GradientBoostingRegressor(n_estimators=300, learning_rate=0.05, max_depth=3, random_state=42) 
                    }

In [21]:
reg_results=[]

for name,model in regression_models.items():
    model.fit(X_train_r,y_train_r)
    preds = model.predict(X_test_r)
    mae = mean_absolute_error(y_test_r,preds)
    mse = mean_squared_error(y_test_r,preds)
    rmse = np.sqrt(mse)
    reg_results.append((name,mae,rmse))

In [22]:
reg_results_df = pd.DataFrame( reg_results, columns= ["Model", "MAE", "RMSE"])

reg_results_df

,Model,MAE,RMSE
0,Linear Regression,1.793677,2.199446
1,Ridge Regression,1.770190,2.106931


#### Initially, I tried out Random Forest Regressor and Gradient Boosting Regressor, but they were both computationally expensive and also, did not scale efficiently on the system. This was mainly due to the high-dimensional and sparse nature of TF-IDF text features. Therefore, Linear Regression model is best for this.

#### The baseline model for regression was Linear Regression. To increase stability on high-dimensional TF-IDF features, Ridge Regression (L2-regularized Linear Regression) was also assessed.

#### Since, Ridge Regressor achieved lower MAE and RMSE, it is chosen as the final regressor model.

In [24]:
final_clf = classification_models["Random Forest"]
final_reg = regression_models["Ridge Regression"]

In [27]:
import os

os.makedirs("models",exist_ok = True)
joblib.dump(final_clf,"models/classifier.pkl")
joblib.dump(final_reg,"models/regressor.pkl")
joblib.dump(vectorizer, "models/tfidf.pkl")
joblib.dump(KEYWORDS, "models/keywords.pkl")

['models/keywords.pkl']

In [28]:
def predict_problem(description,input_desc,output_desc):
    text = clean_text(description + " " + input_desc + " " + output_desc)
    tfidf_vec=vectorizer.transform([text])
    key_vec = np.array([keyword_features(text)])
    num_vec = np.array([numeric_features(text)])
    final_vec = hstack([tfidf_vec,key_vec,num_vec])

    predicted_class = final_clf.predict(final_vec)[0]
    predicted_score = final_reg.predict(final_vec)[0]

    return predicted_class,predicted_score

##### EXAMPLE PROBLEM

In [29]:
predict_problem(
    "Given a graph, find the shortest path using BFS",
    "First line contains N and M",
    "Print the shortest distance"
)


('easy', np.float64(3.361814858279736))